In [1]:
import os
os.environ['PIP_CACHE_DIR'] = '/scratch/ola5/cache'
os.environ['HF_HOME'] = '/scratch/ola5/cache'

In [2]:
%cd Open-Sora-Plan

/scratch/ola5/opensora/Open-Sora-Plan


/scratch/ola5/opensora/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
weight_dtype = torch.bfloat16

from transformers import T5EncoderModel, MT5EncoderModel, AutoTokenizer

In [4]:
text_encoder = MT5EncoderModel.from_pretrained("./models/mt5-xxl", cache_dir='./cache_dir', low_cpu_mem_usage=True, torch_dtype=weight_dtype)
tokenizer = AutoTokenizer.from_pretrained("./models/mt5-xxl", cache_dir='./cache_dir')

text_encoder.eval()
text_encoder = text_encoder.to(device)

/scratch/ola5/opensora/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/scratch/ola5/opensora/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokeniz

In [5]:
print(sum(p.numel() for p in text_encoder.parameters()))

5655173120


In [6]:
prompt = "(best quality), (detailed), Daft Punk dining at a fancy restaurant, eating delicious spaghetti from a plate."
negative_prompt = "lowres, bad anatomy, cropped, worst quality, low quality, jpeg artifacts, signature, watermark, username, blurry"
max_length = 300
num_images_per_prompt = 1
dtype = weight_dtype
batch_size = 1

text_inputs = tokenizer(prompt, padding="max_length", max_length=max_length, truncation=True, add_special_tokens=True, return_tensors="pt")
text_input_ids = text_inputs.input_ids

prompt_attention_mask = text_inputs.attention_mask
prompt_attention_mask = prompt_attention_mask.to(device)

prompt_embeds = text_encoder(text_input_ids.to(device), attention_mask=prompt_attention_mask)
prompt_embeds = prompt_embeds[0]

prompt_embeds = prompt_embeds.to(dtype=text_encoder.dtype, device=device)

bs_embed, seq_len, _ = prompt_embeds.shape
# duplicate text embeddings and attention mask for each generation per prompt, using mps friendly method
prompt_embeds = prompt_embeds.repeat(1, num_images_per_prompt, 1)
prompt_embeds = prompt_embeds.view(bs_embed * num_images_per_prompt, seq_len, -1)
prompt_attention_mask = prompt_attention_mask.view(bs_embed, -1)
prompt_attention_mask = prompt_attention_mask.repeat(num_images_per_prompt, 1)

uncond_tokens = [negative_prompt] * 1
max_length = prompt_embeds.shape[1]
uncond_input = tokenizer(
    uncond_tokens,
    padding="max_length",
    max_length=max_length,
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt",
)
negative_prompt_attention_mask = uncond_input.attention_mask
negative_prompt_attention_mask = negative_prompt_attention_mask.to(device)

negative_prompt_embeds = text_encoder(
    uncond_input.input_ids.to(device), attention_mask=negative_prompt_attention_mask
)
negative_prompt_embeds = negative_prompt_embeds[0]

seq_len = negative_prompt_embeds.shape[1]

negative_prompt_embeds = negative_prompt_embeds.to(dtype=dtype, device=device)

negative_prompt_embeds = negative_prompt_embeds.repeat(1, num_images_per_prompt, 1)
negative_prompt_embeds = negative_prompt_embeds.view(batch_size * num_images_per_prompt, seq_len, -1)

negative_prompt_attention_mask = negative_prompt_attention_mask.view(bs_embed, -1)
negative_prompt_attention_mask = negative_prompt_attention_mask.repeat(num_images_per_prompt, 1)

In [7]:
print(prompt_embeds.shape)
print(prompt_attention_mask.shape)
print(negative_prompt_embeds.shape)
print(negative_prompt_attention_mask.shape)

torch.Size([1, 300, 4096])
torch.Size([1, 300])
torch.Size([1, 300, 4096])
torch.Size([1, 300])


In [8]:
import pickle
prompt_data = (prompt_embeds, prompt_attention_mask, negative_prompt_embeds, negative_prompt_attention_mask)
with open('prompt.pkl', 'wb') as f:
    pickle.dump(prompt_data, f)

In [9]:
with open('prompt.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data[0].shape, data[1].shape)

torch.Size([1, 300, 4096]) torch.Size([1, 300])
